In [2]:
import re
import json
import matplotlib.pyplot as plt
import requests
from PIL import Image
from docx import Document
from docx.shared import Inches, Pt
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from PIL import Image

Download du livre en txt

In [3]:
def Download(url, nom_fichier_sortie):
    reponse = requests.get(url)
    if reponse.status_code == 200:
        with open(nom_fichier_sortie, 'w', encoding='utf-8') as fichier:
            fichier.write(reponse.text)
        print(f"Page web téléchargée avec succès dans {nom_fichier_sortie}")
    else:
        print(f"Échec du téléchargement. Statut HTTP : {reponse.status_code}")

url = "https://www.gutenberg.org/cache/epub/1342/pg1342.txt"
nom_fichier_sortie = "livres.txt"
Download(url, nom_fichier_sortie)

Page web téléchargée avec succès dans livres.txt


Lire je txt et faire un json auteur et titre

In [4]:
def extract_title_and_author(text):
    title_pattern = r"The Project Gutenberg eBook of (.+?)\n"
    author_pattern = r"by\s+([^\n]+)"

    title_match = re.search(title_pattern, text)
    author_match = re.search(author_pattern, text)

    title = title_match.group(1).strip() if title_match else "Title not found"
    author = author_match.group(1).strip() if author_match else "Author not found"

    return title, author

def extract_title_and_author_with_encoding(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()
    except UnicodeDecodeError:
        #problème encodage à gérer
        with open(file_path, 'r', encoding='cp1252') as file:
            text = file.read()

    return extract_title_and_author(text)

def create_json_with_title_and_author(file_path):
    title, author = extract_title_and_author_with_encoding(file_path)
    book_info = {
        "title": title,
        "author": author
    }
    json_file_path = file_path.replace('.txt', '.json')
    with open(json_file_path, 'w') as json_file:
        json.dump(book_info, json_file, indent=4)

    return json_file_path

file_path = 'livres.txt'
json_file_path = create_json_with_title_and_author(file_path)


extraire le chapitre 1 et en faire un txt

In [5]:
def extract_and_save_chapter_one(input_file_path, output_file_path):
    try:
        with open(input_file_path, 'r', encoding='utf-8') as file:
            content = file.read()

        start_index = content.find("Chapter I.]")
        end_index = content.find("[Illustration: M^{r.} & M^{rs.} Bennet")

        if start_index == -1 or end_index == -1:
            return "Chapter markers not found."

        # Extraire le contenu entre les deux index
        chapter_one_content = content[start_index:end_index]

        with open(output_file_path, 'w', encoding='utf-8') as file:
            file.write(chapter_one_content)
        
        return f"Chapter I extracted to {output_file_path}"
    except Exception as e:
        return f"An error occurred: {e}"
    
extract_and_save_chapter_one('livres.txt', 'chapter_one.txt')
chapter_path = 'chapter_one.txt'

compter les mots par paragraphes, un peu galère car c'est des discussions donc j'ai simulé des paragraphes grace aux sauts de lignes de minium 2

In [7]:
def count_sort_paragraphs_and_group_by_dizaine(chapter_path):
    with open(chapter_path, 'r', encoding='utf-8') as file:
        content = file.read()

    paragraphs = content.split('\n\n')

    word_counts = []
    for para in paragraphs:
        word_count = len(para.split())
        word_counts.append(word_count)

    word_counts.sort(reverse=True)

    print("Nombres de mots par paragraphe (du plus long au plus court) :")
    print(word_counts)

    paragraph_count_by_dizaine = {}
    for count in word_counts:
        rounded_count = round(count / 10) * 10
        if rounded_count in paragraph_count_by_dizaine:
            paragraph_count_by_dizaine[rounded_count] += 1
        else:
            paragraph_count_by_dizaine[rounded_count] = 1

    print("\nNombre de paragraphes par tranche de dix mots :")
    for dizaine in sorted(paragraph_count_by_dizaine):
        print(f"{dizaine} mots : {paragraph_count_by_dizaine[dizaine]} paragraphes")

chapter_path = 'chapter_one.txt'
print(count_sort_paragraphs_and_group_by_dizaine(chapter_path))

Nombres de mots par paragraphe (du plus long au plus court) :
[18, 17, 16, 16, 16, 16, 15, 15, 15, 15, 15, 15, 15, 15, 15, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 12, 12, 12, 12, 12, 12, 12, 12, 11, 11, 11, 11, 11, 11, 11, 11, 10, 10, 10, 9, 9, 8, 8, 7, 7, 7, 7, 7, 6, 5, 5, 5, 5, 5, 5, 4, 4, 4, 4, 2, 2, 2, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

Nombre de paragraphes par tranche de dix mots :
0 mots : 57 paragraphes
10 mots : 50 paragraphes
20 mots : 15 paragraphes
None


là je fais le graph en enlevant les erreurs ou y'a 0 mots par paragphes et je le sauvegarde

In [8]:
def plot_non_zero_paragraph_lengths(chapter_path):
    with open(chapter_path, 'r', encoding='utf-8') as file:
        content = file.read()

    paragraphs = content.split('\n\n')

    word_counts = [len(para.split()) for para in paragraphs if len(para.split()) > 0]

    length_counts = {}
    for count in word_counts:
        if count in length_counts:
            length_counts[count] += 1
        else:
            length_counts[count] = 1

    sorted_lengths = sorted(length_counts.keys())
    sorted_counts = [length_counts[length] for length in sorted_lengths]

    plt.bar(sorted_lengths, sorted_counts)
    plt.xlabel('Longueur des paragraphes (nombre de mots)')
    plt.ylabel('Nombre d\'occurrences')
    plt.title('Distribution de la longueur des paragraphes')
    plt.savefig(output_filename)
    plt.close()

output_filename = 'data.png'
plot_non_zero_paragraph_lengths(chapter_path)

télécharger et redimensionner l'image

In [9]:
def download_and_resize_image(url, scaling_factor=0.2, filename='img.png'):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            with open(filename, 'wb') as file:
                file.write(response.content)
            

            with Image.open(filename) as img:
                new_size = (int(img.width * scaling_factor), int(img.height * scaling_factor))
                resized_img = img.resize(new_size)
                resized_img.save(filename)

            return f"L'image a été téléchargée, redimensionnée à 20% de sa taille originale, et sauvegardée sous le nom '{filename}'."
        else:
            return "Erreur lors du téléchargement de l'image : la requête a échoué."
    except Exception as e:
        return f"Une erreur est survenue lors du téléchargement ou de la redimension de l'image : {e}"
    
result = download_and_resize_image('https://m.media-amazon.com/images/M/MV5BMTA1NDQ3NTcyOTNeQTJeQWpwZ15BbWU3MDA0MzA4MzE@._V1_.jpg')

Collage de mon coeur en noir et blanc sur l'image 1
je les converts sinon ça passe pas et je génère une nouvelle image

In [10]:
imagebase = Image.open('img.png')
imagecollage = Image.open('hearth.png')

imagebase=imagebase.convert("RGBA")
imagecollage=imagecollage.convert("RGBA")

imagebase.alpha_composite(imagecollage, dest=(25,25))
imagebase.save('collage.png')
imagebase.show()

j'utilise le json et l'image ainsi que des choses écrites directement dans le code pour générer mon document word

In [11]:
with open('livres.json', 'r') as file:
    book_info = json.load(file)

doc = Document()

report_author = "Sambre Vandercoilden"

title_para = doc.add_paragraph()
title_run = title_para.add_run(book_info['title'])
title_run.font.size = Pt(24)
title_para.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

author_para = doc.add_paragraph(f"Author: {book_info['author']}")
author_para.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

doc.add_picture('collage.png', width=Inches(3))
doc.paragraphs[-1].alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

doc.add_page_break()

doc.add_paragraph("La campagne anglaise à la fin du XVIIIe siècle. Mrs. Bennet et son mari sont ravis d'apprendre qu'un jeune homme fortuné - et célibataire - vient de s'installer dans le manoir voisin. Désargentés, les Bennet se font fort de marier l'une de leurs cinq filles au nouvel arrivant... Ce dernier ne tarde pas à s'éprendre de la belle Jane, l'aînée de la famille, lors d'un bal de campagne. \nGraphique montrant la répartition du nombre de mots dans le premier chapitre du livre. \nLe maximum étant de 18 et le minimum de 1 parmis 82 paragraphes. \nL'image de couverture du livre est également incluse.")

image_path = 'data.png'
with Image.open(image_path) as img:
    original_width, original_height = img.size
    new_width, new_height = original_width * 1.1, original_height * 1.1

width_in_inches = new_width / 96
height_in_inches = new_height / 96

doc.add_picture(image_path, width=Inches(width_in_inches), height=Inches(height_in_inches))
doc.paragraphs[-1].alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

report_author_para = doc.add_paragraph(report_author)
report_author_para.alignment = WD_PARAGRAPH_ALIGNMENT.RIGHT

# ON SAVE ET C EST FINI
doc.save('pride_and_prejudice_report.docx')
